# Mosaic Sentinel-2 images

A mosaic is a combination or merge of two or more images. With `rasterio`, you can create a single raster from multiple raster by mosaicking them together.

<figure class="image">
  <img src="mosaic_ex.png" alt="mosaic" width="500">
</figure>

In [26]:
import glob, os
import numpy as np
import geopandas as gpd
import rasterio
import rasterio.plot
from rasterio.merge import merge
import matplotlib.pyplot as plt
from pathlib import Path

**Set parameters**

In [29]:
tile_list = ['31UFS','31UFR']
tile_list = ['31UFS','31UFR','31UGS','31UGR']


tile_list_str = '_'.join(tile_list)

nodata_val = -10000

**Set directory**

In [30]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_nb        = '99'

# Directory for all work files
work_path = f'{computer_path}STUDENTS/GROUP_{grp_nb}/TP/'


masked_path = f'{work_path}3_L2A_MASKED_4tiles/'
mosaic_path = f'{work_path}5_L2A_MOSAIC_{tile_list_str}/'

print(mosaic_path)

Path(mosaic_path).mkdir(parents=True, exist_ok=True)

/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/5_L2A_MOSAIC_31UFS_31UFR_31UGS_31UGR/


In [31]:
list_im_masked_one_tile = glob.glob(f'{masked_path}*{tile_list[0]}*_ROI_SCL.tif')
list_im_masked_all_tile = glob.glob(f'{masked_path}*_ROI_SCL.tif')

print(f'There are {len(list_im_masked_one_tile)} images')

There are 51 images


In [32]:
for im_file_1 in list_im_masked_one_tile:

    im_to_mosaic_list = [im_file_1]
    
    # Get date of image
    date_1 = os.path.basename(im_file_1)[7:7+8]

    # Get tile of image
    tile_1 = os.path.basename(im_file_1)[0:6]

    # Get spectral band
    band_1 = os.path.basename(im_file_1)[23:26]


    for im_file_2 in list_im_masked_all_tile:
        
        # Get date of image
        date_2 = os.path.basename(im_file_2)[7:7+8]

        # Get tile of image
        tile_2 = os.path.basename(im_file_2)[0:6]

        # Get spectral band
        band_2 = os.path.basename(im_file_2)[23:26]

        if date_1 == date_2:
            if band_1 == band_2:
                if tile_1 != tile_2:
                    print(f'Date : {date_1} - Band : {band_1}')
                    print('There are at least 2 images to mosaic')
            
                    im_to_mosaic_list.append(im_file_2)
    
    
    if len(im_to_mosaic_list) > 1:

        print(im_to_mosaic_list)
    
        mosaic_file = f'{mosaic_path}{os.path.basename(im_file_1)[:-4]}_mosaic_{tile_list_str}.tif'

        print(mosaic_file)

        src_files_to_mosaic = []
        for file in im_to_mosaic_list:
            src = rasterio.open(file)
            src_files_to_mosaic.append(src)
        
        src_files_to_mosaic[0].meta

        mosaic, out_trans = merge(src_files_to_mosaic)

        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans})

        with rasterio.open(mosaic_file, "w", **out_meta) as dst:
            dst.write(mosaic)
        
        for src in src_files_to_mosaic:
            src.close()

Date : 20200805 - Band : B11
There are at least 2 images to mosaic
Date : 20200805 - Band : B11
There are at least 2 images to mosaic
Date : 20200805 - Band : B11
There are at least 2 images to mosaic
['/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/3_L2A_MASKED_4tiles/T31UFS_20200805T104031_B11_10m_ROI_SCL.tif', '/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/3_L2A_MASKED_4tiles/T31UFR_20200805T104031_B11_10m_ROI_SCL.tif', '/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/3_L2A_MASKED_4tiles/T31UGS_20200805T104031_B11_10m_ROI_SCL.tif', '/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/3_L2A_MASKED_4tiles/T31UGR_20200805T104031_B11_10m_ROI_SCL.tif']
/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/5_L2A_MOSAIC_31UFS_31UFR_31UGS_31UGR/T31UFS_20200805T104031_B11_10m_ROI_SCL_mosaic_31UFS_31UFR_31UGS_31UGR.tif
Date : 20200805 - Band : B04
There are at least 2 images to mosaic
Date : 20200805 - Band : B04
There are at least 2 images to mosaic
Date : 20200805 - B